In [1]:
from datetime import datetime
import uuid
import pandas as pd
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData
import numpy as np
from app.database.queries import insert_data
from typing import List, Dict
import logging
import logging
from typing import Dict, List
from sqlalchemy import MetaData
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
from app.database.connect import dbconnect

🔃 Connecting
✅ Connected to pg server


In [2]:
logger = logging.getLogger(__name__)  # Use a named logger

In [3]:
logging.basicConfig(level=logging.INFO)
RECIPE = "../../app/data/processed/recipe_cleaned.json"
INGREDIENTS = "../../app/data/processed/ingredients_AI.json"
MEAL = "../../app/data/processed/meal_cleaned.json"
MEALTORECIPE = "../../app/data/processed/mealtorecipe.json" 

In [4]:
logger.info("PREPARING DATA")
data: pd.DataFrame = pd.read_json(MEAL)
data = data.replace({np.nan: None, '': None, ' ': None})
data.head()

INFO:__main__:PREPARING DATA


,meal_id,name,description,created_at,source,rating,votes
0,a6213618-a4b0-4b70-a132-a0cf4763bd02,Plantain Pancakes with Spicy Maple Syrup and P...,These delicious plantain pancakes are topped o...,2024-09-25 00:03:22,https://www.yummymedley.com/plantain-pancakes-...,0.0,0
1,2ba7aba9-0be6-4c4c-8a1f-f7c5c2947b80,Afang Soup Recipe,"Afang soup, eaten by Efik and Ibibio people of...",2024-09-25 00:03:23,https://www.yummymedley.com/nigerian-afang-soup/,5.0,8
2,456099e2-0242-4462-8a50-49398ade9cf6,Fonio Porridge/Acha Pudding: A Super-Grain Bre...,"Fonio is a gluten-free, nutritious, super grai...",2024-09-25 00:03:24,https://www.yummymedley.com/fonio-porridge-ach...,5.0,1
3,6d50d4e7-dc64-47e3-8871-c848fe0240fc,Mixed Seafood Coconut Fried Rice Recipe,In this delicious mixed seafood coconut fried ...,2024-09-25 00:03:25,https://www.yummymedley.com/fishermans-fried-r...,5.0,11
4,82d85396-2e38-4802-99e4-f3d405cd7304,Peppered Snail Recipe,"The spicy, chewy goodness of peppered snail co...",2024-09-25 00:03:26,https://www.yummymedley.com/peppered-snail/,5.0,4


In [5]:
data["updated_at"] = data["created_at"]
data = data.rename(columns={"meal_id": "id"})

In [6]:
data["status"] = "PUBLISHED"
data["video_url"] = "Not Available"

In [7]:
data = data[["id", "name", "description", "status", "created_at", "updated_at", "source", "video_url"]]
len(data)

220

In [8]:
df_cleaned = data.drop_duplicates(subset='name', keep='first')
df_cleaned = df_cleaned.reset_index().drop(columns="index")

In [9]:
data = df_cleaned.copy()
data_dict = data.to_dict(orient='records')

In [10]:
# insert_data(data=data_dict, schema="Meal")

In [11]:
meal_to_recipe = pd.read_json(MEALTORECIPE)
meal_to_recipe.head(3)

,meal_id,recipe_id
0,a6213618-a4b0-4b70-a132-a0cf4763bd02,2c2703b4-ed39-468d-a09b-4dda480e3bc7
1,2ba7aba9-0be6-4c4c-8a1f-f7c5c2947b80,099498aa-d5d1-4be5-be81-8525014cd5da
2,456099e2-0242-4462-8a50-49398ade9cf6,9534b335-0657-4da0-abc7-c2a58272f25a


In [12]:
filtered_df = meal_to_recipe[meal_to_recipe['meal_id'].isin(df_cleaned['id'])]
recipe_to_meal = filtered_df.rename(columns={"meal_id": "A", "recipe_id": "B"})
recipe_to_meal.head()

,A,B
0,a6213618-a4b0-4b70-a132-a0cf4763bd02,2c2703b4-ed39-468d-a09b-4dda480e3bc7
1,2ba7aba9-0be6-4c4c-8a1f-f7c5c2947b80,099498aa-d5d1-4be5-be81-8525014cd5da
2,456099e2-0242-4462-8a50-49398ade9cf6,9534b335-0657-4da0-abc7-c2a58272f25a
3,6d50d4e7-dc64-47e3-8871-c848fe0240fc,8d378ba0-3b46-4a23-b349-a1a183bbc4ec
4,82d85396-2e38-4802-99e4-f3d405cd7304,ac3857b2-7a42-4e88-af04-35361b550f2b


In [13]:
data_rm = recipe_to_meal.copy()
data_dict = data_rm.to_dict(orient='records')
# insert_data(data=data_dict, schema="_MealToRecipe")

In [ ]:
CATEGORY = "../../app/data/processed/recipes_AI.json"
cat = pd.read_json(CATEGORY)
recipe_filtered = cat[cat['recipe_id'].isin(data_rm['B'])]
recipe_filtered

,meal_type,recipe_id,alternative_meal_type,category,course_type
0,breakfast,2c2703b4-ed39-468d-a09b-4dda480e3bc7,NaN,NaN,NaN
1,,099498aa-d5d1-4be5-be81-8525014cd5da,,,"Main Course, Side Dish"
2,breakfast,9534b335-0657-4da0-abc7-c2a58272f25a,NaN,NaN,NaN
3,dinner,8d378ba0-3b46-4a23-b349-a1a183bbc4ec,NaN,NaN,NaN
4,,ac3857b2-7a42-4e88-af04-35361b550f2b,,,Appetizer
...,...,...,...,...,...
143,lunch/dinner,b4d8589f-3948-4e51-a172-e9827403871e,NaN,NaN,NaN
144,Soup,ce359911-75fa-42a5-8a1f-d9e3f0ce480b,,Appetizer,Side Dish
145,snack,ada2bd7c-f09e-4581-a6e1-131342da397d,NaN,NaN,NaN
146,dinner,fbfe493f-23df-4a5b-bb88-9fc1446ea001,lunch,chicken,main course
